# core

> Stores the main class to boost a estimator

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from sklearn.ensemble import RandomForestRegressor
import typing as t
import numpy as np
from scipy.spatial import cKDTree
from dataclasses import dataclass, field

In [ ]:
#| export
Estimator = t.Any
Columns = t.Union[t.Literal["*"], t.List[int]]


@dataclass
class KNNSpatialBooster:
    """
    A KNN Spatial Booster.

    You can use it with any model. It uses the training dataset to improve
    the spatial perception of the model adding more features. Remember to
    use warm_start=True if your model is a scikit-learn model.

    Read more in the `KNNSpatialBooster` docs.

    Parameters
    ----------

    n_neighbors : int, default=5
        The number of neighbors to use as feature. Also known as "k".
    
    temperature : float, default=.2
        If temperature=0, exactly "k" nearest neighbors are selected. If
        temperature is higher, other neighbors can be selected.
    
    n_loops : int, default=50
        Quantity of .fit() calls. If temperature>0, will create a new
        dataset foreach loop.
    
    verbose : bool, default=False
        If set to True, will print the current loop.

    estimator : Estimator, default=RandomForestRegressor()
        Any Estimator as defined by scikit-learn with warm_start=True.

    estimator_output_1d_array : bool, default=True
        Used to be compatible with estimators that needs 1d arrays in `.fit()`.

    spatial_features : Columns, default="*"
        Define which columns should be used as coordinates for KNN.
    
    remove_target_spatial_cols : bool, default=False
        Set to True if you believe the model should not use the original
        spatial features in the training process.
    
    remove_neighbor_spatial_cols : bool, default=True
        Set to True if you believe the model should not use the neighbors
        spatial features in the training process.
    """
    n_neighbors: int = field(default=5)
    temperature: float = field(default=0.2)
    n_loops: int = field(default=50)
    verbose: bool = field(default=False)
    estimator: Estimator = field(default=RandomForestRegressor())
    estimator_output_1d_array: bool = field(default=True)
    spatial_features: Columns = field(default="*")
    remove_target_spatial_cols: bool = field(default=False)
    remove_neighbor_spatial_cols: bool = field(default=True)
    
    def fit(self, X: np.ndarray, Y: np.ndarray) -> None:
        """
        Fit model for X and Y.

        Parameters
        ----------

        X : np.ndarray
            Shape must be (number of samples,) or (number of samples, number_of_features).
        Y : np.ndarray
            Shape must be (number of samples,) or (number of samples, number of outputs).

        """
        if len(X.shape) < 1 or len(X.shape) > 2:
            raise Exception(f"Bad X shape: {X.shape}")

        if len(Y.shape) < 1 or len(Y.shape) > 2:
            raise Exception(f"Bad Y shape: {Y.shape}")

        self.X = X if len(X.shape) == 2 else X.reshape((X.shape[0], 1))
        self.Y = Y if len(Y.shape) == 2 else Y.reshape((Y.shape[0], 1))

        self.spatial_cols = (
            list(range(self.X.shape[1]))
            if self.spatial_features == "*"
            else self.spatial_features
        )

        neighbors = KNNSpatialBooster.get_neighbors(
            self.X,
            self.Y,
            self.X,
            self.spatial_cols,
            self.n_neighbors,
            remove_first_neighbor=True,
            remove_neighbor_spatial_cols=self.remove_neighbor_spatial_cols
        )

        boosted_X = KNNSpatialBooster.enrich_dataset(
            neighbors,
            self.X,
            self.spatial_cols,
            self.n_neighbors,
            self.temperature,
            remove_target_spatial_cols=self.remove_target_spatial_cols,
        )

        for i in range(self.n_loops):
            print(f"Running loop #{i}")

            self.estimator.fit(
                boosted_X, Y.ravel() if self.estimator_output_1d_array else Y
            )
            if self.temperature > 0 and i < self.n_loops - 1:
                boosted_X = KNNSpatialBooster.enrich_dataset(
                    neighbors,
                    self.X,
                    self.spatial_cols,
                    self.n_neighbors,
                    self.temperature,
                    remove_target_spatial_cols=self.remove_target_spatial_cols,
                )


    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predict for X.

        Parameters
        ----------

        X : np.ndarray
            Shape must be (number of samples,) or (number of samples, number_of_features).

        Returns
        -------

        Y : np.ndarray
            Predicted value for given X.
        """

        if len(X.shape) < 1 or len(X.shape) > 2:
            raise Exception(f"Bad X shape: {X.shape}")

        X = X if len(X.shape) == 2 else X.reshape((X.shape[0], 1))

        neighbors = KNNSpatialBooster.get_neighbors(
            self.X,
            self.Y,
            X,
            self.spatial_cols,
            self.n_neighbors,
            remove_neighbor_spatial_cols=self.remove_neighbor_spatial_cols
        )

        boosted_X = KNNSpatialBooster.enrich_dataset(
            neighbors,
            X,
            self.spatial_cols,
            self.n_neighbors,
            0,
            remove_target_spatial_cols=self.remove_target_spatial_cols,
        )

        return self.estimator.predict(boosted_X)

    def score(self, X: np.ndarray, Y: np.ndarray) -> float:
        """
        Calculate score for X and Y. It uses the estimator score function.

        Parameters
        ----------

        X : np.ndarray
            Shape must be (number of samples,) or (number of samples, number_of_features).
        Y : np.ndarray
            Shape must be (number of samples,) or (number of samples, number of outputs).

        Returns
        -------

        score : float
            Estimator score.
        """

        if len(X.shape) < 1 or len(X.shape) > 2:
            raise Exception(f"Bad X shape: {X.shape}")

        if len(Y.shape) < 1 or len(Y.shape) > 2:
            raise Exception(f"Bad Y shape: {Y.shape}")

        X = X if len(X.shape) == 2 else X.reshape((X.shape[0], 1))
        Y = Y if len(Y.shape) == 2 else Y.reshape((Y.shape[0], 1))

        neighbors = KNNSpatialBooster.get_neighbors(
            self.X,
            self.Y,
            X,
            self.spatial_cols,
            self.n_neighbors,
            remove_neighbor_spatial_cols=self.remove_neighbor_spatial_cols
        )

        boosted_X = KNNSpatialBooster.enrich_dataset(
            neighbors,
            X,
            self.spatial_cols,
            self.n_neighbors,
            0,
            remove_target_spatial_cols=self.remove_target_spatial_cols,
        )

        return self.estimator.score(
            boosted_X, Y.ravel() if self.estimator_output_1d_array else Y
        )
    
    @staticmethod
    def vectorized_choice(probablities: np.ndarray, n: int, k: int) -> np.ndarray:
        cumulative = probablities.cumsum()
        random_matrix = np.random.rand(n, probablities.shape[0])
        q = cumulative.reshape(1, -1) >= random_matrix
        indexes = (~q).argsort(axis=1)[:, range(k)]
        return indexes

    @staticmethod
    def random_first_k(n: int, k: int, temperature: float) -> np.ndarray:
        if temperature < 0.01:
            temperature = 0.01
        distribution = np.array([np.exp(-i) for i in range(2*k)]) / temperature
        adjusted_distribution = np.exp(distribution) / np.sum(np.exp(distribution))
        matrix = KNNSpatialBooster.vectorized_choice(adjusted_distribution, n, k)
        return np.sort(matrix, axis=1)

    @staticmethod
    def get_neighbors(
        base_X: np.ndarray,
        base_y: np.ndarray,
        target_X: np.ndarray,
        spatial_cols: t.List[int],
        n_neighbors: int,
        remove_first_neighbor: bool = False,
        remove_neighbor_spatial_cols: bool = True,
    ) -> np.ndarray:
        first_index = 1 if remove_first_neighbor else 0
        k = n_neighbors + first_index
        btree = cKDTree(base_X[:, spatial_cols])
        distances, indexes = btree.query(target_X[:, spatial_cols], k=2*k)

        neighbors_cols = [
            i
            for i in range(base_X.shape[1])
            if not (remove_neighbor_spatial_cols and (i in spatial_cols))
        ]

        neighbors_tuples = [
            (
                base_X[idx.ravel(), :][:, neighbors_cols],
                base_y[idx.ravel(), :],
                np.reciprocal(d + 1).reshape((len(d), 1)),
            )
            for d, idx in zip(np.hsplit(distances, 2*k), np.hsplit(indexes, 2*k))
        ]

        neighbors = np.array([np.hstack(neighbors_tuple) for neighbors_tuple in neighbors_tuples][first_index:]).T
        return neighbors


    @staticmethod
    def enrich_dataset(
        neighbors: np.ndarray,
        target_X: np.ndarray,
        spatial_cols: t.List[int],
        n_neighbors: int,
        temperature: float,
        remove_target_spatial_cols: bool = False,
    ) -> np.ndarray:
        neighbors_indexes = KNNSpatialBooster.random_first_k(target_X.shape[0], n_neighbors, temperature)
        selected_neighbors = np.hstack([np.take_along_axis(feature, neighbors_indexes, axis=1) for feature in neighbors])

        taget_cols = [
            i
            for i in range(target_X.shape[1])
            if not (remove_target_spatial_cols and (i in spatial_cols))
        ]

        return np.hstack([target_X[:, taget_cols], selected_neighbors])

In [ ]:
show_doc(KNNSpatialBooster.fit)

---

### KNNSpatialBooster.fit

>      KNNSpatialBooster.fit (X:numpy.ndarray, Y:numpy.ndarray)

Fit model for X and Y.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | ndarray | Shape must be (number of samples,) or (number of samples, number_of_features). |
| Y | ndarray | Shape must be (number of samples,) or (number of samples, number of outputs). |
| **Returns** | **None** |  |

In [ ]:
show_doc(KNNSpatialBooster.predict)

---

### KNNSpatialBooster.predict

>      KNNSpatialBooster.predict (X:numpy.ndarray)

Predict for X.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | ndarray | Shape must be (number of samples,) or (number of samples, number_of_features). |
| **Returns** | **ndarray** | **Predicted value for given X.** |

In [ ]:
show_doc(KNNSpatialBooster.score)

---

### KNNSpatialBooster.score

>      KNNSpatialBooster.score (X:numpy.ndarray, Y:numpy.ndarray)

Calculate score for X and Y. It uses the estimator score function.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | ndarray | Shape must be (number of samples,) or (number of samples, number_of_features). |
| Y | ndarray | Shape must be (number of samples,) or (number of samples, number of outputs). |
| **Returns** | **float** | **Estimator score.** |

In [ ]:
KNNSpatialBooster.random_first_k(10, 3, 0.5)

array([[0, 2, 3],
       [0, 2, 3],
       [0, 1, 2],
       [0, 3, 4],
       [1, 2, 3],
       [0, 1, 2],
       [1, 3, 4],
       [0, 1, 3],
       [0, 2, 3],
       [0, 1, 2]])

In [ ]:
KNNSpatialBooster.get_neighbors(
    np.array([
        [0, 1, 10],
        [0, 1, 15],
        [0, 1, 17],
        [0, 2, 20],
        [0, 2, 25],
        [0, 2, 27],
        [2, 0, 30],
        [2, 0, 35],
        [2, 0, 37],
    ]),
    np.array([
        [20],
        [30],
        [35],
        [40],
        [50],
        [55],
        [60],
        [70],
        [75],
    ]),
    np.array([
        [1, 1, 40],
        [2, 1, 50],
    ]),
    [0, 1],
    3,
    remove_first_neighbor=True
)

array([[[10.        , 15.        , 27.        , 35.        ,
         30.        , 25.        , 20.        ],
        [30.        , 35.        , 17.        , 10.        ,
         15.        , 27.        , 25.        ]],

       [[20.        , 30.        , 55.        , 70.        ,
         60.        , 50.        , 40.        ],
        [60.        , 70.        , 35.        , 20.        ,
         30.        , 55.        , 50.        ]],

       [[ 0.5       ,  0.5       ,  0.41421356,  0.41421356,
          0.41421356,  0.41421356,  0.41421356],
        [ 0.5       ,  0.5       ,  0.33333333,  0.33333333,
          0.33333333,  0.30901699,  0.30901699]]])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()